# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
#load csv into dataframe
csv_path = "../cities.csv"
city_weather_df = pd.read_csv(csv_path)

#drop rows with missing data
city_weather_df = city_weather_df.dropna()

city_weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
#configure gmaps with API key 
gmaps.configure(g_key)

locations = city_weather_df[["Lat", "Lng"]]
humidity = city_weather_df["Humidity"].astype(float)

In [30]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#create a dataframe for the specific weather criteria
weather_df = pd.DataFrame(city_weather_df, columns =["City", "Cloudiness", "Max Temp", "Wind Speed"])

#specify the ideal weather for each criteria
max_temp = (weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] <=80)
wind_speed = weather_df["Wind Speed"] < 10
cloudiness = weather_df["Cloudiness"] == 0

weather_df[max_temp & wind_speed & cloudiness]

,City,Cloudiness,Max Temp,Wind Speed
70,ponta do sol,0,73.68,5.32
88,cidreira,0,76.56,5.50
138,morondava,0,79.08,9.06
139,vaini,0,78.90,1.59
176,nador,0,75.20,6.93
297,mogok,0,72.24,2.77
444,birjand,0,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df = pd.DataFrame(city_weather_df, columns = ["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
hotel_df 

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [33]:
#list hotel names
hotel_list = []

x = 0

for city in hotel_df['City']:
    
    target_coordinates = f"{hotel_df['Lat'][x]}, {hotel_df['Lng'][x]}"
    x += 1
    
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params).json()


KeyError: 7

In [34]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params).json()
response["results"]

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 27.732345, 'lng': 109.18979},
   'viewport': {'northeast': {'lat': 27.73390922989272,
     'lng': 109.1912712298927},
    'southwest': {'lat': 27.73120957010728, 'lng': 109.1885715701073}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
  'name': 'Tongren Grand Metro Park Hotel',
  'photos': [{'height': 3024,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103022163065277942595">Andrew Goh</a>'],
    'photo_reference': 'ATtYBwIX5dGzaJ44VQNyNnYY4d-UK2yrGx1cPVZeUHOK2TnucVcEJFVJnNVnLmjxfrtds1B8PkaYR8zOXLvubJA4v7Fn3ClWJVe9jrc2duVRabv3tf_r6_I9Hho58CVBMmuxr2eDp-N2gYUjja6FfGvDX__EGEtl6dvx485j4PaIIoJk4CbA',
    'width': 4032}],
  'place_id': 'ChIJyzlfwYeVlzYR9eh3Bhy98Yk',
  'plus_code': {'compound_code': 'P5JQ+WW Bijiang District, Tongren, Guizhou, China',
   'global_code': '7PVFP5JQ+WW'},
  'rating': 4.7,
  'reference': 'ChIJyzlfwYeVlzYR9eh3Bhy98Yk',
  'sco

In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layers(markers)

# Display figure
fig

AttributeError: 'Figure' object has no attribute 'add_layers'